In [1]:
from pathlib import Path, PurePosixPath
import csv
import sys
import re
import numpy as np

np.array([]).reshape((3, 1))

ValueError: cannot reshape array of size 0 into shape (3,1)

In [4]:
"  sd df     df ".split()

['sd', 'df', 'df']

In [4]:
import pandas as pd
tu = {'some': [34], 'sd': [3]}
data = pd.DataFrame(tu)

In [5]:
data

,some,sd
0,34,3


In [6]:
anime = pd.DataFrame()

In [11]:
anime

""


In [12]:
anime.append({'d': 23, 'sdf': 2334}, ignore_index=True)

,d,sdf
0,23.0,2334.0


In [1]:
import numpy as np

In [17]:
np.random.seed(234123)
np.round(np.random.rand(4) * 10000000, 0)
for i in range(0, 20):
    np.random.seed(i * i * 100 - i * 23 + 23)
    a = np.round(np.random.rand(4) * 10000000, 0)
    print("RMULT ", end='')
    for g in a:
        print(f"{int(g)}, ", end='')
    print()



RMULT 5172979, 9469626, 7654598, 2823958, 
RMULT 5434049, 2783694, 4245176, 8447761, 
RMULT 1184841, 5209490, 5218728, 8211005, 
RMULT 5175444, 8128924, 365192, 5126712, 
RMULT 4716132, 8172305, 4052273, 3877967, 
RMULT 5007714, 2420424, 4546119, 9585569, 
RMULT 7663868, 1079002, 149992, 9107074, 
RMULT 2097794, 1256779, 624108, 7823031, 
RMULT 4495781, 6788391, 3897248, 5182450, 
RMULT 1992136, 5280163, 6309680, 4218898, 
RMULT 417483, 6267724, 9756208, 8844493, 
RMULT 8038058, 9563251, 3286474, 2488633, 
RMULT 2711404, 6698774, 3942332, 4356797, 
RMULT 9801535, 257489, 7621974, 304627, 
RMULT 4599893, 832274, 9083731, 5641024, 
RMULT 8605589, 341040, 1480367, 3610990, 
RMULT 2991505, 8676789, 1773092, 3143011, 
RMULT 795147, 5645931, 4912870, 5135887, 
RMULT 7341205, 6471129, 3721273, 787848, 
RMULT 1907465, 2884323, 5191840, 3608852, 
